<b><h4>Feature Selection is the process of selecting a subset of features for Model Construction.</h4></b>

<h4><b>There are some great benefits of doing feature selection in Kaggle Competitions.</b></h4><br>
1. Shorter training times.

2. Less memory usage.

3. Simplification of models and data.

4. Avoid curse of dimensionality.

5. Avoid noisy and irrelevant features.

<br>**The BorutaShap package, as the name suggests, combines the [Boruta feature selection algorithm](https://www.jstatsoft.org/article/view/v036i11) with the [SHAP](https://christophm.github.io/interpretable-ml-book/shap.html) technique.<br><br>**

**This notebook demonstrates how to apply Boruta-SHAP Feature Selection on [Kaggle : American Express - Default Prediction](https://www.kaggle.com/competitions/amex-default-prediction).**

**I used the data generated by this [notebook](https://www.kaggle.com/code/susnato/amex-data-preprocesing-feature-engineering) using this [dataset](https://www.kaggle.com/datasets/raddar/amex-data-integer-dtypes-parquet-format).**

<font color='red'>**<h3>If you like this notebook then please upvote</h3>**

## Install requirements

In [ ]:
!pip install BorutaShap -q
!pip install xgboost==1.6.1 -q

## Imports

In [ ]:
import os
import gc
import glob
import tqdm
import numpy as np
import pandas as pd

## Load data

In [ ]:
SEED = 42

train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
train_labels['customer_ID'] = train_labels['customer_ID'].apply(lambda x: int(x[-16:], 16)).astype(np.int64)
train_labels = train_labels.set_axis(train_labels['customer_ID'])
train_labels = train_labels.drop(['customer_ID'], axis=1)

train_pkls = sorted(glob.glob('../input/amex-data-preprocesing-feature-engineering/train_data_*'))
train_y = sorted(glob.glob('../input/amex-data-preprocesing-feature-engineering/train_y_*.npy'))
test_pkls = sorted(glob.glob('../input/amex-data-preprocesing-feature-engineering/test_data_*'))

In [ ]:
from sklearn.model_selection import train_test_split

train_df = pd.read_pickle(train_pkls[0])
print(train_pkls[0])
for i in train_pkls[1:]:
    print(i)
    train_df = train_df.append(pd.read_pickle(i))
    gc.collect()
    
y = train_labels.loc[train_df.index.values].values.astype(np.int8)
train_df = train_df.drop(['D_64_1', 'D_66_0', 'D_68_0'], axis=1)

X_train, X_val, y_train, y_val = train_test_split(train_df, y, 
                                                  stratify=y, test_size=0.25, 
                                                  random_state=42, shuffle=True)
print(X_train.shape,  y_train.shape, X_val.shape, y_val.shape)
del train_df, y
_ = gc.collect()

## Load and run the algorithm

In [ ]:
import xgboost as xgb
from BorutaShap import BorutaShap

xgb_clf = xgb.XGBClassifier(
        n_estimators = 1500,  
        max_depth = 3,  
        learning_rate = 0.05,  
        subsample = 0.7,  
        colsample_bytree = 0.5,  
        min_child_weight = 5,  
        reg_alpha = 0.5,  
        reg_lambda = 2,  
        tree_method ='gpu_hist',
        predictor = 'gpu_predictor',
        random_state = 42)

Feature_Selector = BorutaShap(model = xgb_clf, 
                              importance_measure='shap', 
                              classification=True)
Feature_Selector.fit(X=X_train.iloc[:75000], y=y_train[:75000], 
                     n_trials=50, random_state=SEED)

del xgb_clf
_ = gc.collect()

In [ ]:
Feature_Selector.plot(which_features='accepted', figsize=(16,12))

In [ ]:
Feature_Selector.plot(which_features='tentative', figsize=(16,12))

**If we have `tentative` features then we can either run the algorithm for more `trials` or we can use `Feature_Selector.TentativeRoughFix()`. It will sort out those `tentative` features into useful and not useful ones.**

In [ ]:
Feature_Selector.TentativeRoughFix()

## Now let's test new features

**But first we need to fit the model with all the data so that we can draw a baseline.**

In [ ]:
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def amex_metric(preds: np.ndarray, target: np.ndarray) -> float:
    indices = np.argsort(preds)[::-1]
    preds, target = preds[indices], target[indices]

    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_mask = cum_norm_weight <= 0.04
    d = np.sum(target[four_pct_mask]) / np.sum(target)

    weighted_target = target * weight
    lorentz = (weighted_target / weighted_target.sum()).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    n_pos = np.sum(target)
    n_neg = target.shape[0] - n_pos
    gini_max = 10 * n_neg * (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)

    g = gini / gini_max
    return 0.5 * (g + d)

In [ ]:
xgb_clf_full = xgb.XGBClassifier(
        n_estimators = 1500,  
        max_depth = 3,  
        learning_rate = 0.05,  
        subsample = 0.7,  
        colsample_bytree = 0.5,  
        min_child_weight = 5,  
        reg_alpha = 0.5,  
        reg_lambda = 2,  
        tree_method ='gpu_hist',
        predictor = 'gpu_predictor',
        random_state = SEED)

xgb_clf_full.fit(X_train, y_train,
                 eval_set=[(X_train, y_train), (X_val, y_val)],
                 verbose=100)

metric_score_full = amex_metric(target=y_val.reshape(-1, ), 
                                preds=xgb_clf_full.predict_proba(X_val)[:, 1].reshape(-1, ))
print("The model scored : {} after using the whole set of features.".format(metric_score_full))

del xgb_clf_full
_ = gc.collect()

**Let's now train the model with only selected features**

In [ ]:
selected_cols = Feature_Selector.accepted
print("There are only {} features selected".format(len(selected_cols)))

xgb_clf_selected = xgb.XGBClassifier(
        n_estimators = 1500,  
        max_depth = 3,  
        learning_rate = 0.05,  
        subsample = 0.7,  
        colsample_bytree = 0.5,  
        min_child_weight = 5,  
        reg_alpha = 0.5,  
        reg_lambda = 2,  
        tree_method ='gpu_hist',
        predictor = 'gpu_predictor',
        random_state = SEED)

xgb_clf_selected.fit(X_train[selected_cols], y_train,
                 eval_set=[(X_train[selected_cols], y_train), 
                           (X_val[selected_cols], y_val)],
                 verbose=100)

metric_score_selected = amex_metric(target=y_val.reshape(-1, ), 
                                    preds=xgb_clf_selected.predict_proba(X_val[selected_cols])[:, 1].reshape(-1, ))
print("The model scored : {} after using only : {} features.".format(metric_score_selected, len(selected_cols)))

del xgb_clf_selected
_ = gc.collect()

**We can get more performance by using more data.<br>
I used Colab Pro to train the model on almost 300000 rows and here is the result.**

In [ ]:
selected_cols_colab = np.load('../input/shapxgboost/accepted_features_XGBOOST.npy').tolist()
print("There are only {} features selected".format(len(selected_cols_colab)))

xgb_clf_selected_colab = xgb.XGBClassifier(
        n_estimators = 1500,  
        max_depth = 3,  
        learning_rate = 0.05,  
        subsample = 0.7,  
        colsample_bytree = 0.5,  
        min_child_weight = 5,  
        reg_alpha = 0.5,  
        reg_lambda = 2,  
        tree_method ='gpu_hist',
        predictor = 'gpu_predictor',
        random_state = SEED)

xgb_clf_selected_colab.fit(X_train[selected_cols_colab], y_train,
                 eval_set=[(X_train[selected_cols_colab], y_train), 
                           (X_val[selected_cols_colab], y_val)],
                 verbose=100)

metric_score_selected_colab = amex_metric(target=y_val.reshape(-1, ), 
                                    preds=xgb_clf_selected_colab.predict_proba(X_val[selected_cols_colab])[:, 1].reshape(-1, ))
print("The model scored : {} after using only : {} features.".format(metric_score_selected_colab, len(selected_cols_colab)))

del xgb_clf_selected_colab
_ = gc.collect()

**As you can see, we are getting almost the same result with 116 features. And the previous result was not too far off despite having used very little data.**

## Conclusion

* **Thie Features are for XGBOOST. But we can use any model as long as it has `fit` and `predict` method.**

* **In this example we used a very thin slice of data and still managed to get almost the same result as before.**

* **We can save a lot of memory by only using the `accepted` features. because Kaggle Notebook only allows us to load limited amount of features into memory.**

* **As this competition goes on, more and more people will find useful and interesting features, we may use this approach then to find the relevent ones in those.**

## Reference

* https://github.com/Ekeany/Boruta-Shap

* https://medium.com/analytics-vidhya/is-this-the-best-feature-selection-algorithm-borutashap-8bc238aa1677

* https://www.kaggle.com/code/carlmcbrideellis/feature-selection-using-the-boruta-shap-package/notebook